In [9]:
import json
import pandas as pd
import requests
import mysql.connector 
import config
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd
)

print(cnx)

In [10]:
cursor = cnx.cursor()

In [5]:
from mysql.connector import errorcode
db_name = 'Movies'

def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

Database Movies does not exists.
Database Movies created successfully.


In [7]:
db = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = "Movies")

cursor = db.cursor()

In [ ]:
api_key = '__'
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }
url_params = { action=query?
             }

In [45]:
# response = requests.get("http://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&rvsection=0&titles=pizza")
response = requests.get("https://en.wikipedia.org/w/api.php?action=opensearch&search=list-of-American-films-of-2019&format=json")

In [46]:
response

<Response [200]>

In [47]:
response.json()

['list-of-American-films-of-2019',
 ['List of American films of 2019',
  'List of American films of 2018',
  'List of American films of 2014',
  'List of American films of 2015',
  'List of American films of 2017',
  'List of American films of 2016',
  'List of American films of 2013',
  'List of American films of 2009',
  'List of American films of 2010',
  'List of American films of 2012'],
 ['This is a list of American films that were released, or are scheduled to release in 2019.',
  'This is a list of American films that were released in 2018.',
  'This is a list of American films released in 2014.',
  'This is a list of American films released in 2015.',
  'This is a list of American films that were released in 2017.',
  'This is a list of American films released in 2016.',
  'This is a list of American films released in 2013.',
  'A list of American films released on 2009.',
  'A list of American films released in 2010.',
  'This is a list of American films released in 2012.'],


In [49]:
!pip install wikipedia

  Stored in directory: /Users/cblessen/Library/Caches/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [50]:
import wikipedia

In [59]:
section = wikipedia.WikipediaPage(title = 'List of American films of 2019')

In [65]:
import wikipedia
import requests
from bs4 import BeautifulSoup
import time
import numpy as np

# first pull the HTML from the page that links to all of the pages with the links.
# in this case, this page gives the links list pages of sci-fi films by decade.
# just go to https://en.wikipedia.org/wiki/Lists_of_science_fiction_films
# to see what I'm pulling from.
html = requests.get('https://en.wikipedia.org/wiki/List_of_American_films_of_2019')

#turn the HTML into a beautiful soup text object
b = BeautifulSoup(html.text, 'lxml')
# create an mpty list where those links will go.
links = []

# in this case, all of the links we're in a '<li>' brackets.
for i in b.find_all(name = 'li'):
    # pull the actual link for each one
    for link in i.find_all('a', href=True):
        links.append(link['href'])
# the above code ends up pulling more links than I want,
# so I just use the ones I want
links = links[1:11]
# each link only returns something like 'wiki/List_of_science_fiction_films_of_the_1920s'
# so I add the other part of the URL to each.
decade_links = ['https://en.wikipedia.org' + i for i in links]

# create two new lists, one for the title of the page, 
# and one for the link to the page
film_titles = []
film_links = []
# for loop to pull from each decade page with list of films.
# look at https://en.wikipedia.org/wiki/List_of_science_fiction_films_of_the_1920s
# to follow along as an exampe
for decade in decade_links:
    print(f'Collecting films from {decade}')
    html = requests.get(decade)
    b = BeautifulSoup(html.text, 'lxml')
    # get to the table on the page
    for i in b.find_all(name='table', class_='wikitable'):
        # get to the row of each film
        for j in i.find_all(name='tr'):
            #get just the title cell for each row.
            # contains the title and the URL
            for k in j.find_all(name='i'):
                # get within that cell to just get the words
                for link in k.find_all('a', href=True):
                    # get the title and add to the list
                    film_titles.append(link['title'])
                    # get the link and add to that list
                    film_links.append(link['href'])
    #be a conscientious scraper and pause between scrapes
    time.sleep(1)
print(f'Number of Film Links Collected: {len(film_links)}')
print(f'Number of Film Titles Collected: {len(film_titles)}')
# remove film links that don't have a description page on Wikipedia
new_film_links = [i for i in film_links if 'redlink' not in i]
# same goes for titles
new_film_titles = [i for i in film_titles if '(page does not exist)' not in i]
print(f'Number of Film Links with Wikipedia Pages: {len(new_film_links)}')
print(f'Number of Film Titles with Wikipedia Pages: {len(new_film_titles)}')
#use this list to fetch from the API
title_links = list(zip(new_film_titles, new_film_links))

ConnectionError: HTTPSConnectionPool(host='en.wikipedia.orghttps', port=443): Max retries exceeded with url: //en.wikipedia.org/w/index.php?title=Template:American_film_list&action=edit (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11eeb1208>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))